<a href="https://colab.research.google.com/github/mohsinziabutt/Applied-AI-Challenge-2/blob/main/NLP_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing dataset**

In [48]:
import pandas as pd

tweets_data = pd.read_csv('https://raw.githubusercontent.com/mohsinziabutt/Applied-AI-Challenge-2/main/train.csv')
tweets_data.head()

,Id,Text,Sentiment
0,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
1,088c60f138,my boss is bullying me...,negative
2,9642c003ef,what interview! leave me alone,negative
3,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative
4,6e0c6d75b1,2am feedings for the baby are fun when he is a...,positive


# **Checking if there is any sentiment in the column other than negative and positive**

In [49]:
tweets_data.pivot_table(columns=['Sentiment'], aggfunc='size')

Sentiment
negative    7781
positive    8582
dtype: int64

# **Data Pre-Processing**

**1. Defining the columns we need**

In [50]:
X=tweets_data['Text']
y=tweets_data['Sentiment']

**2. Removing Stop Words**

In [51]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

stop_words=stopwords.words('english')
stemmer=PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**3. Cleaning the data we required**

In [52]:
import re

cleaned_data=[]
for i in range(len(X)):
   tweet=re.sub('[^a-zA-Z]',' ',X.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data.append(tweet)

cleaned_data

['sooo sad miss san diego',
 'boss bulli',
 'interview leav alon',
 'son put releas alreadi bought',
 'feed babi fun smile coo',
 'journey wow u becam cooler hehe possibl',
 'realli realli like song love stori taylor swift',
 'sharpi run danger low ink',
 'want go music tonight lost voic',
 'uh oh sunburn',
 'ok tri plot altern speak sigh',
 'sick past day thu hair look wierd didnt hat would look http tinyurl com mnf kw',
 'back home gonna miss everi one',
 'play ghost onlin realli interest new updat kirin pet metamorph third job wait dragon pet',
 'free fillin app ipod fun im addict',
 'sorri',
 'way malaysia internet access twit',
 'juss came backk berkeleyi omg madd fun havent minut whassqoodd',
 'went sleep power cut noida power back work',
 'go home seen new twitter design quit heavenli',
 'hope unni make audit fight dahy unni',
 'consol got bmi test hahaha say obes well much unhappi minut',
 'funni cute kid',
 'born rais nyc live texa past year still miss ny',
 'soooooo sleeeeepi

**4. Bag of Words Approach using Count Vectorizer**

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X_fin=cv.fit_transform(cleaned_data).toarray()

# **Converting the sentiments into numbers**

In [54]:
sentiment_ordering = ['negative', 'positive']
y = y.apply(lambda x: sentiment_ordering.index(x))

y

0        0
1        0
2        0
3        0
4        1
        ..
16358    1
16359    0
16360    0
16361    1
16362    1
Name: Sentiment, Length: 16363, dtype: int64

# **Defining the model we are going to use**

In [55]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

# **Splitting the dataset into train and test data and fitting the model**

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_fin, y, test_size=0.3, random_state=100)
model.fit(X_train, y_train)

MultinomialNB()

# **Performing the predictions and generating the classification report**

In [57]:
y_pred=model.predict(X_test)
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      2398
           1       0.85      0.87      0.86      2511

    accuracy                           0.85      4909
   macro avg       0.85      0.85      0.85      4909
weighted avg       0.85      0.85      0.85      4909



# **Importing the testing file to generate the sentiments against new tweets**

In [58]:
#import the csv file with tweets to be labelled
new_tweets = pd.read_csv("https://raw.githubusercontent.com/mohsinziabutt/Applied-AI-Challenge-2/main/test.csv")
new_tweets = new_tweets[["Text"]]

new_tweets["Sentiment"] = ""
new_tweets.head()

,Text,Sentiment
0,Shanghai is also really exciting (precisely -...,
1,"Recession hit Veronique Branquinho, she has to...",
2,happy bday!,
3,http://twitpic.com/4w75p - I like it!!,
4,that`s great!! weee!! visitors!,


# **Converting the tweets into Bag of Words using Count Vectorizer**

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X_fin2=cv.fit_transform(new_tweets['Text']).toarray()

# **Predicting the sentiments for new tweets**

In [60]:
new_sentiments = model.predict(X_fin2)
new_tweets["Sentiment"] = new_sentiments

# **Converting the Sentiments back from numeric to string**

In [62]:
sentiments = {1:'positive', 0:'negative'}
new_tweets["Sentiment"] = [sentiments[item] for item in new_tweets["Sentiment"]]

# **Saving final results as CSV file**

In [63]:
new_tweets = new_tweets[['Text', "Sentiment"]]
new_tweets.to_csv('new_tweets_predictions.csv', header=True, index=False)